<a href="https://colab.research.google.com/github/nitsundon/AIML/blob/main/XGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import plotly.express as px

In [2]:
url="https://raw.githubusercontent.com/nitsundon/AIML/refs/heads/main/amr2.csv"
df = pd.read_csv(url)


In [3]:
df=df[['datetime','sum']].copy()


In [5]:
# fig = px.line(df, x="datetime", y="sum",
#               title="State Demand Over Time (15-Min Blocks)",
#               labels={"Timestamp": "Time", "Demand_MW": "Demand (MW)"},
#               template="plotly_dark")  # Optional: Use "seaborn" or "simple_white" theme

# # Show interactive graph
# fig.show()

In [4]:
df["datetime"] = pd.to_datetime(df["datetime"].copy(), errors="coerce")
df["Hour"] = df["datetime"].dt.hour
df["DayOfWeek"] = df["datetime"].dt.dayofweek
df["Month"] = df["datetime"].dt.month
df["Quarter"] = df["datetime"].dt.quarter
df["Year"] = df["datetime"].dt.year
df["DayOfyear"] = df["datetime"].dt.dayofyear
df["DayOfmonth"] = df["datetime"].dt.day
df["WeekOfYear"] = df["datetime"].dt.isocalendar().week
df["BlockNumber"] = 1+df["datetime"].dt.hour * 4 + df["datetime"].dt.minute // 15

In [6]:
df1=df[~(((df['datetime']>'2021-01-01 00:00:00') & (df['datetime']<'2021-12-31 23:59:00')) | ((df['datetime']>'2024-06-09 23:59:00') & (df['datetime']<'2024-06-16 23:59:00')))]

In [8]:
# import plotly.express as px
# fig = px.line(df1, x="datetime", y="sum",
#               title="State Demand Over Time (15-Min Blocks)",
#               labels={"Timestamp": "Time", "Demand_MW": "Demand (MW)"},
#               template="plotly_dark")  # Optional: Use "seaborn" or "simple_white" theme

# # Show interactive graph
# fig.show()

In [7]:
scada_data_url="https://raw.githubusercontent.com/nitsundon/AIML/refs/heads/main/scada.csv"
df2= pd.read_csv(scada_data_url)

In [8]:
df2.columns=['datetime','demand']
df2["datetime"] = pd.to_datetime(df2["datetime"].copy(), errors="coerce")

In [9]:
df2["datetime"] = pd.to_datetime(df2["datetime"].copy(), errors="coerce")
df2["Hour"] = df2["datetime"].dt.hour
df2["DayOfWeek"] = df2["datetime"].dt.dayofweek
df2["Month"] = df2["datetime"].dt.month
df2["Quarter"] = df2["datetime"].dt.quarter
df2["Year"] = df2["datetime"].dt.year
df2["DayOfyear"] = df2["datetime"].dt.dayofyear
df2["DayOfmonth"] = df2["datetime"].dt.day
df2["WeekOfYear"] = df2["datetime"].dt.isocalendar().week
df2["BlockNumber"] = 1+df2["datetime"].dt.hour * 4 + df2["datetime"].dt.minute // 15

In [10]:
df3=df1.merge(df2[['datetime','demand']], on='datetime', how='outer')

In [13]:

# fig = px.line(df3, x="datetime", y=["sum","demand"],
#               title="State Demand Over Time (15-Min Blocks)",
#               labels={"Timestamp": "Time", "Demand_MW": "Demand (MW)"},
#               template="plotly_dark")  # Optional: Use "seaborn" or "simple_white" theme

# # Show interactive graph
# fig.show()

In [11]:
df4=df3[~df3['demand'].isnull()]
df5=df3[df3['demand'].isnull()]

In [12]:
X=df4[[ 'Hour', 'DayOfWeek', 'Month', 'Quarter', 'Year',
       'DayOfyear', 'DayOfmonth', 'WeekOfYear', 'BlockNumber', 'sum']]
Y=df4['demand']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, random_state=45)


In [14]:
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)

# Train Model
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)

# Evaluate Performance
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 1007.1824552650901


In [15]:
import numpy as np

# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE: {mape:.2f}%')

MAPE: 5.41%


In [16]:
X1=df5[[ 'Hour', 'DayOfWeek', 'Month', 'Quarter', 'Year',
       'DayOfyear', 'DayOfmonth', 'WeekOfYear', 'BlockNumber', 'sum']]

In [17]:
Y1= model.predict(X1)

In [18]:
X1=X1.reset_index().drop(columns="index")

In [19]:
merge1=X1.merge(pd.DataFrame(Y1), left_index=True, right_index=True)

In [20]:
merge1=merge1.rename(columns={0:"demand"})

In [ ]:
df6=pd.merge(df3,merge1,on=['Hour', 'DayOfWeek', 'Month', 'Quarter', 'Year',
       'DayOfyear', 'DayOfmonth', 'WeekOfYear', 'BlockNumber','sum'],how="left")

In [ ]:
# fig = px.line(df6.iloc[55500:105500], x="datetime", y=["sum","demand_x","demand_y"],
#               title="State Demand Over Time (15-Min Blocks)",
#               labels={"Timestamp": "Time", "Demand_MW": "Demand (MW)"},
#               template="simple_white")  # Optional: Use "seaborn" or "simple_white" theme

# # Show interactive graph
# fig.show()

In [ ]:
df7=df6.copy()
df7['demand_x']=df7['demand_x'].fillna(0)
df7['demand_y']=df7['demand_y'].fillna(0)

In [ ]:
df7['scadaDemand']=df7['demand_x']+df7['demand_y']

In [ ]:
df7=df7.drop(columns=['sum','demand_x','demand_y'])

In [ ]:
df7['lag_1'] = df7['scadaDemand'].shift(1)
df7['lag_2'] = df7['scadaDemand'].shift(96)
df7['lag_3'] = df7['scadaDemand'].shift(672)
split_ratio = 0.8
split_index = int(len(df7) * split_ratio)

train, test = df7[:split_index], df7[split_index:]

In [ ]:
test

In [ ]:
X=train[[ 'Hour', 'DayOfWeek', 'Month', 'Quarter', 'Year',
       'DayOfyear', 'DayOfmonth', 'WeekOfYear', 'BlockNumber','lag_1','lag_2','lag_3']]
Y=train['scadaDemand']
X_test=test[[ 'Hour', 'DayOfWeek', 'Month', 'Quarter', 'Year',
       'DayOfyear', 'DayOfmonth', 'WeekOfYear', 'BlockNumber','lag_1','lag_2','lag_3']]
y_test=test['scadaDemand']
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)

# Train Model
model.fit(X,Y)

# Make Predictions
y_pred = model.predict(X_test)

# Evaluate Performance
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

In [ ]:
df8=test.reset_index().merge(pd.DataFrame(y_pred), left_index=True, right_index=True)
df8.rename(columns={0:"Predicted"},inplace=True)

In [ ]:
# fig = px.line(df8, x=df8['datetime'], y=["Predicted","scadaDemand"],
#               title="State Demand Over Time (15-Min Blocks)",
#               labels={"Timestamp": "Time", "Demand_MW": "Demand (MW)"},
#               template="simple_white")  # Optional: Use "seaborn" or "simple_white" theme

# # Show interactive graph
# fig.show()